# Future:
Try implementing a dual memory. one with regular game states and the other with death states so that it can be somewhat ephasized during training and when the bot succeeds at beating the game it will still remember the game over moves.

In [1]:
import gym
from tqdm import tqdm
from collections import deque
import random
import torch
import torch.nn as nn
import math
import numpy as np
import time
from jupyterplot import ProgressPlot
torch.manual_seed(0)
random.seed(0)


In [2]:
class Mem:
    def __init__(self, cap):
        self.que = deque([], maxlen=cap)
    
    def __len__(self):
        return len(self.que)
    
    def sample(self, bs):
        return random.sample(self.que, bs)
    
    def push(self, data):
        self.que.append(data)

In [3]:
def action(state, eps):
    if random.random() < eps: #pick random action
        return torch.randint(0,2,(), device=device)
        return torch.randperm(2, device=device)
    else:
        with torch.no_grad():
            a = torch.argmax(pol(state).squeeze())
        return a

In [4]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.flat = nn.Flatten()
        self.linear1 = nn.Linear(4, 10)
        self.linear3 = nn.Linear(10, 10)
        self.linear4 = nn.Linear(10, 2)
        self.act = nn.Mish()
        
    def forward(self, x):
        x = self.flat(x)
        x = self.linear1(x)
        x = self.act(x)
        x = self.linear3(x)
        x = self.act(x)
        x = self.linear4(x)
        return x
    

    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pol = Qnet().to(device)
target = Qnet().to(device)
target.load_state_dict(pol.state_dict())
target.eval()

optimizer = torch.optim.Adam(pol.parameters())
loss = nn.SmoothL1Loss()
#loss = nn.MSELoss()

mem = Mem(int(1e6))

In [5]:
env = gym.make('CartPole-v1')
env.seed(42)

[42]

In [6]:
EPOCHS = 10000
EPS_MIN = 0.05
EPS_MAX = 0.95
EPS_MOMENT = 0.9
EPS_DEC = 200
GAMMA = 0.999
BS = 512
TARGET_UPDATE = 10

In [7]:
def opt():
    if len(mem) < BS:
        return False
    pol.train()
    
    #transpose
    data = list(zip(*mem.sample(BS)))

    s = torch.cat(data[0])
    a = torch.stack(data[1])
    ns = torch.cat(data[2])
    r = torch.stack(data[3])

    q = pol(s).gather(1, a.view(-1,1))

    with torch.no_grad():
        qg = torch.zeros((len(r), 1), device=device)
        mask = r != 0
        qg[mask] = GAMMA * target(ns[mask]).max(1, keepdim=True)[0] + r[mask].reshape((-1,1))

    optimizer.zero_grad()

    l = loss(q, qg)
    l.backward()
    for param in pol.parameters():
        param.grad.data.clamp_(-1,1)
    optimizer.step()

    return l.item()

In [ ]:
pp = ProgressPlot(line_names=['duration'])
#ppr = ProgressPlot(line_names=['rewards'])

ema = 0
for i in range(EPOCHS):
    env.render()
    pol.eval()
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float, device=device).unsqueeze(0)
    c = 0
    tr = 0
    best = 0
    avgloss = 0
    losscount = 1
    while True:
        env.render()
        c += 1
        
        #eps = (EPS_MAX-EPS_MIN)/(1+np.exp((-c+ema)/20)) + EPS_MIN
        eps = EPS_MIN + (EPS_MAX - EPS_MIN) * math.exp(-1. * i / EPS_DEC)
        a = action(state, 0.05)
        ns, r, d, _ = env.step(a.item())
        ns = torch.tensor(ns.copy(), dtype=torch.float, device=device).unsqueeze(0)
                
        tr += r
        r = torch.tensor(r, dtype=torch.float, device=device)
        l = opt()
        if l:
            avgloss = avgloss + (l-avgloss)/(losscount)
            losscount += 1
        if d:
            ema = EPS_MOMENT * ema + (1-EPS_MOMENT) * c
            mem.push((state, a, ns, torch.tensor(0, device=device)))
            break
        #mem.push((state, a[i], ns, r))
        mem.push((state, a, ns, r))
        state = ns
    if i % TARGET_UPDATE == 0:
        target.load_state_dict(pol.state_dict())
    pp.update(c)
    print(avgloss, '\t\t', end='\r')
    #ppr.update(float(best))
pp.finalize()
#ppr.finalize()

# If you wanan just use the pretrained models
the file names are w.pt and winner.pt. both will achieve max scores that you can watch.

In [ ]:
#pol.load_state_dict(torch.load('w.pt'))

In [31]:
for i in range(1000):
    state = env.reset()
    state = torch.tensor(state, dtype=torch.float, device=device).unsqueeze(0)
    d = False
    while not d:
        env.render()
        a = action(state, 0)
        ns, r, d, _ = env.step(a.item())
        ns = torch.tensor(ns.copy(), dtype=torch.float, device=device).unsqueeze(0)
        time.sleep(1/20)
        state = ns

KeyboardInterrupt: 

In [15]:
file = 'savefile.pt'
torch.save(pol.state_dict(), file)